In [1]:
import torch
import torchvision

In [2]:
n_classes = 4
batch_size = 4
n_epochs = 5
path_best_model = "objet_model.pth"

In [4]:
path_dataset = "U:\\PROJET_3A\\dataset_objets" 
ratio_train_val = 2/3

dataset = torchvision.datasets.ImageFolder(
    path_dataset,
    torchvision.transforms.Compose([
        torchvision.transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        torchvision.transforms.Resize(256),
        torchvision.transforms.CenterCrop(224),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

n_train = int(ratio_train_val * len(dataset))
n_valid = len(dataset) - n_train

train_dataset, valid_dataset = torch.utils.data.random_split(dataset, [n_train, n_valid])

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
)

valid_loader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=batch_size,
    shuffle=True,
)

In [4]:
"""
path_train_dataset = "C:\\Users\\alexi\\PROJET_3A\\imagenette2-160\\train"
path_valid_dataset = "C:\\Users\\alexi\\PROJET_3A\\imagenette2-160\\val"

train_dataset =  torchvision.datasets.ImageFolder(
    path_train_dataset,
    torchvision.transforms.Compose([
        torchvision.transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        torchvision.transforms.Resize((224, 224)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

valid_dataset = torchvision.datasets.ImageFolder(
    path_valid_dataset,
    torchvision.transforms.Compose([
        torchvision.transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        torchvision.transforms.Resize((224, 224)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
)

valid_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
)
"""

FileNotFoundError: [WinError 3] Le chemin d’accès spécifié est introuvable: 'C:\\Users\\alexi\\PROJET_3A\\imagenette2-160\\train'

In [5]:
model = torchvision.models.alexnet(pretrained=True)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, n_classes)

In [6]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

best_accuracy = 0.0

for epoch in range(n_epochs):
    for i, (images, labels) in enumerate(train_loader) :
        optimizer.zero_grad()
        outputs = model(images)
        loss = torch.nn.functional.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
        print('epoch %d batch %d/%d' % (epoch, (i + 1), len(train_loader)))
    test_error_count = 0.0
    for images, labels in valid_loader:
        outputs = model(images)
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))
    test_accuracy = 1.0 - float(test_error_count) / float(len(valid_dataset))
    print('test accuracy : %f' % test_accuracy)
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), path_best_model)
        best_accuracy = test_accuracy

epoch 0 batch 1/23
epoch 0 batch 2/23
epoch 0 batch 3/23
epoch 0 batch 4/23
epoch 0 batch 5/23
epoch 0 batch 6/23
epoch 0 batch 7/23
epoch 0 batch 8/23
epoch 0 batch 9/23
epoch 0 batch 10/23
epoch 0 batch 11/23
epoch 0 batch 12/23
epoch 0 batch 13/23
epoch 0 batch 14/23
epoch 0 batch 15/23
epoch 0 batch 16/23
epoch 0 batch 17/23
epoch 0 batch 18/23
epoch 0 batch 19/23
epoch 0 batch 20/23
epoch 0 batch 21/23
epoch 0 batch 22/23
epoch 0 batch 23/23
test accuracy : 0.787234
epoch 1 batch 1/23
epoch 1 batch 2/23
epoch 1 batch 3/23
epoch 1 batch 4/23
epoch 1 batch 5/23
epoch 1 batch 6/23
epoch 1 batch 7/23
epoch 1 batch 8/23
epoch 1 batch 9/23
epoch 1 batch 10/23
epoch 1 batch 11/23
epoch 1 batch 12/23
epoch 1 batch 13/23
epoch 1 batch 14/23
epoch 1 batch 15/23
epoch 1 batch 16/23
epoch 1 batch 17/23
epoch 1 batch 18/23
epoch 1 batch 19/23
epoch 1 batch 20/23
epoch 1 batch 21/23
epoch 1 batch 22/23
epoch 1 batch 23/23
test accuracy : 0.680851
epoch 2 batch 1/23
epoch 2 batch 2/23
epoch 2 ba